In [1]:
import sys
import os
 
root_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..'))
sys.path.insert(0, root_dir)

from probe_mark.dataset_Della.get_img import create_folders,get_img,make_folder,get_img_label,get_image
# import sys
# import os
 
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

In [7]:
import pandas as pd
# from get_img import create_folders,get_img,make_folder,get_img_label,get_image

import os

from datetime import datetime
nowdir = os.getcwd()# 現在目錄
nowpath=root_dir+"\\data\\processed"
current_date = datetime.now()# 抓取現在的日期
formatted_date = current_date.strftime("%Y%m%d")# 格式化日期為所需格式

folder_path=make_folder(path=nowpath, folder_name='rowdata',date=formatted_date)


已建立資料夾 d:\github\Probe-Mark\data\processed\rowdata_20240807


In [27]:

folder_name = 'Artificial_VR29_D4070_thimble_10g'
base_output_dir=make_folder(path=folder_path, folder_name=folder_name)
# 讀取 Excel 檔案中的特定 sheet
file_path = root_dir+'\\data\\raw\\'+folder_name+'\\picture.xlsx'
sheet_name = '主要'
# 讀取指定 sheet，並跳過第一行，同時指定新的欄位名稱
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1, names=['name', 'img'])
img_name_list=df['name']  # 資料夾名稱列表
# 建立資料夾儲存圖片
created_folders = create_folders(base_output_dir, img_name_list)
get_img(file_path, sheet_name, created_folders,'original')

資料夾已存在 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g
資料夾 Die1-7-1-10g-f 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-7-1-10g-f
資料夾 Die1-7-2-10g 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-7-2-10g
資料夾 Die1-7-3-10g 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-7-3-10g
資料夾 Die1-14-1-10g-f 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-14-1-10g-f
資料夾 Die1-14-2-10g-f 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-14-2-10g-f
資料夾 Die1-14-3-10g-f 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die1-14-3-10g-f
資料夾 Die2-7-1-10g 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240807\Artificial_VR29_D4070_thimble_10g\Die2-7-1-10g
資料夾 Die2-7-2-10g 已建立於 d:\githu

In [28]:

# 讀取 Excel 檔案中的特定 sheet
sheet_name = '體積面積量測'
# # 建立資料夾儲存圖片
get_img(file_path, sheet_name, created_folders,'red_mark')


圖片提取和儲存完成。


In [29]:
for i in range(len(created_folders)):
    image_path1 = created_folders[i] + '\\original.png'
    image_path2 = created_folders[i] + '\\red_mark.png'
    save_path = created_folders[i] + '\\label.png'
    get_img_label(image_path1, image_path2, save_path)